In [1]:
import json
from googleapiclient.discovery import build
import googleapiclient.errors
import pandas as pd
import os
#if os.path.isfile(checkpoint):

In [2]:
api_key = "None"

#use the build() function to create a service object
youtube = build("youtube", "v3", developerKey = api_key)

#print(json.dumps(response,sort_keys=True,indent=3))

In [12]:
#put in a search phrase
numres=25
searchres = youtube.search().list(part="snippet", maxResults=numres, q="Eric Clapton").execute()
#print(searchres["items"][10])

#list to hold ids which we will use to get tags
vidids = []
for i in range(len(searchres["items"])):
    try:
        vidids.append(searchres["items"][i]["id"]["videoId"])
    except KeyError:
        continue

print(len(vidids))

23


In [43]:
#I guess we start simple - we'll save the tags and the title and use those to classify
df = []
#parse and take only data we want
for i in range(len(vidids)):
    response = youtube.videos().list(part="snippet,statistics", id=vidids[i]).execute()
    try:
        df.append(pd.DataFrame({"title": response["items"][0]["snippet"]["title"],\
                           "tags": [response["items"][0]["snippet"]["tags"][:5]],\
                          "likes": response["items"][0]["statistics"]["likeCount"],\
                          "dislikes": response["items"][0]["statistics"]["dislikeCount"],\
                          "views": response["items"][0]["statistics"]["viewCount"],\
                            "Broad Tag": "Music"}))
    except KeyError:         #some videos do not have tags apparently
        continue
if os.path.isfile("vids.pkl"):
    vids = pd.read_pickle("vids.pkl")
    new = pd.concat(df)
    vids = pd.concat([vids,new], ignore_index=True)
    vids.to_pickle("vids.pkl")
else:
    vids = pd.concat(df)
